<a href="https://colab.research.google.com/github/solanods/ProjetosFaculdade/blob/main/grupo_4_entrega_etapa2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Identificação**: Solano Cruz Júnior (2919585), Rogério Dourado Felix (2892220), Newton Douglas da Silva Nascimento (2733408), Raimundo de Sousa Lima Filho (2709691)

---





# Tema: Natural Language Processing (NPL) para análise de sentimentos em postagens do Twitter

O uso de inteligência artificial e machine learning para atividades de mineração de textos da web se apresenta como uma riquíssima fonte de insights para a ciência de dados. Nesse contexto, frameworks de aprendizado de máquina como Sklearn, Keras e Tensorflow têm dominado a cena, realizando desde tarefas simples até a criação de autoresponders, bots, classificadores de texto e vários sistemas de diálogo que determinam o significado das frases. 

Um framework bastante poderoso e com uma curva de aprendizagem um pouco mais curta em relação aos já citados é a biblioteca Spacy. Escrita em Python, possui suporte para mais de 60 idiomas. De código aberto, foi projetada para construir sistemas de extração de informações ou de compreensão de linguagem natural. Também foi desenvolvida para uso em produção e fornece uma API concisa e fácil de usar.


> **Objetivo**

Esse projeto tem como objetivo treinar um modelo capaz de avaliar se uma determinada frase apresenta sentimento POSITIVO ou NEGATIVO. 

> **Especificação Técnica**


Dataset: Para desenvolvimento desse projeto, será utilizado o dataset
denominado Train50, disponível em:  https://github.com/solanods/ProjetosFaculdade/blob/main/Train50.csv.



> **Formato:** A base de dado está em formato CSV. Train 50.csv possui 50mil registros e 5 colunas. 

> **Métodos de Pŕe-processamento:** Será criada uma função em Python para a limpeza e pré-processamentos textuais

Letras minúsculas: serão priorizadas no case das strings.<br>
Nome do usuário: será removido junto do caractere (@)<br>
URLs: remoção<br>
Espaços em branco: remoção<br>
Emoticons: substituidos pelas strings "emocaopostiva", "emocaonegativa"<br>
Stop words: remoção<br>
Lematização: aplicação<br>
Pontuações: remoção<br>



> **Tarefa de Aprendizado:** 

Serão utilizados métodos e funções do Sklearn a saber:



CountVectorizer e TfidfVectorizer: Convertem uma coleção de documentos de texto em uma matriz de contagens de token

accuracy_score: calcula a precisão do subconjunto 

TransformerMixin: Ajustar aos dados e transformá-los.

Pipeline:  usado para encadear vários estimadores em um. Isso é útil, pois geralmente há uma sequência fixa de etapas no processamento dos dados, por exemplo, seleção de recursos, normalização e classificação.

LinearSVC: classes capaz de realizar classificação binária e multiclasse em um conjunto de dados.





 








IMPORTAÇÃO E INSTALAÇÃO DAS BIBLIOTECAS

In [ ]:
!pip install -q spacy==2.2.3 

     |████████████████████████████████| 10.4 MB 7.6 MB/s 
     |████████████████████████████████| 2.2 MB 38.9 MB/s 


In [ ]:
#atualização da biblioteca Spacy para português
!python3 -m spacy download pt


     |████████████████████████████████| 21.2 MB 1.2 MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-py3-none-any.whl size=21186281 sha256=4beb4e3fb1b41d30ab5030f200d152420ce5111bd881e6ce696158d658dbf83e
  Stored in directory: /tmp/pip-ephem-wheel-cache-y5j38kv_/wheels/c3/f9/0c/5c014a36941a00f5df5fc0756cb961d7c457a978e697a6ce3b
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [ ]:
import pandas as pd
import string
import spacy
import random
import seaborn as sns
import numpy as np
import re

In [ ]:
#carregar os dados
df = pd.read_csv('/content/Train50.csv', delimiter=';')

In [ ]:
df.shape

(50000, 5)

In [ ]:
df.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1050785521201541121,@Laranjito76 A pessoa certa para isso seria o ...,Fri Oct 12 16:29:25 +0000 2018,1,:)
1,1050785431955140608,"@behin_d_curtain Para mim, é precisamente o co...",Fri Oct 12 16:29:04 +0000 2018,1,:)
2,1050785401248645120,Vou fazer um video hoje... estou pensando em f...,Fri Oct 12 16:28:56 +0000 2018,1,:)
3,1050785370982547461,"aaaaaaaa amei tanto essas polaroids, nem sei e...",Fri Oct 12 16:28:49 +0000 2018,1,:)
4,1050785368902131713,Valoriza o coração do menininho que vc tem. El...,Fri Oct 12 16:28:49 +0000 2018,1,:)


In [ ]:
#base de dados com 5000 registros (tweets)
#sentimento negativo label 0
#sentimento postivo label 1

df['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [ ]:
# o conjunto de dados possui duas colunas do tipo inteiro e três colunas do tipo string
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          50000 non-null  int64 
 1   tweet_text  50000 non-null  object
 2   tweet_date  50000 non-null  object
 3   sentiment   50000 non-null  int64 
 4   query_used  50000 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.9+ MB


In [ ]:
#eliminamos as colunas que não vamos usar
df.drop(['id', 'tweet_date', 'query_used'], axis = 1, inplace=True)


In [ ]:
df.head()

,tweet_text,sentiment
0,@Laranjito76 A pessoa certa para isso seria o ...,1
1,"@behin_d_curtain Para mim, é precisamente o co...",1
2,Vou fazer um video hoje... estou pensando em f...,1
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1
4,Valoriza o coração do menininho que vc tem. El...,1


In [ ]:
#confirmar se não há valores nulos no dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  50000 non-null  object
 1   sentiment   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


# Função para pré-processamento dos textos

## Serão eliminados de cada registro da base de dados

###Letras maiúsculas
###Nome do usuário (@)
###URLs
###Espaços em branco
###Emoticons
###Stop words
###Lematização
###Pontuações

In [ ]:
# carregamento  do objeto spacy em português
nlp = spacy.load('pt')
nlp

In [ ]:
# construir uma lista de stop words a serem filtradas
stop_words = spacy.lang.pt.stop_words.STOP_WORDS

Stop words (ou palavras de parada – tradução livre) são palavras que podem ser consideradas irrelevantes para o conjunto de resultados a ser exibido em uma busca realizada em uma search engine. Exemplos: as, e, os, de, para, com, sem, foi.

In [ ]:
stop_words

In [ ]:
 # Lematização diz respeito ao nível léxico das palavras
 # Vejamos um exemplo
nlp = spacy.load('pt')
frase = ('Pedro viajou de avião saindo de Santos, após louvar os santos, até Belém do Pará e retornou há três dias.')
tokens = nlp(frase)
for token in tokens:
  print(token.text, "|", token.lemma_)

Pedro | Pedro
viajou | viajar
de | de
avião | avião
saindo | sair
de | de
Santos | Santos
, | ,
após | após
louvar | louvar
os | o
santos | santo
, | ,
até | até
Belém | Belém
do | do
Pará | Pará
e | e
retornou | retornar
há | haver
três | três
dias | dia
. | .


In [ ]:
def preprocessamento(texto):
  # manter tudo em letras minúsculas
  texto = texto.lower()

  # eliminar nome do usuário
  texto = re.sub(r"@[A-Za-z0-9$-_@.&+]+", ' ', texto)

  # eliminar urls
  texto = re.sub(r"https?://[A-Za-z0-9./]+", ' ', texto)

  # eliminar espaços em branco
  texto = re.sub(r" +", ' ', texto)

  # substituir emoticons por texto
  lista_emocoes = {':)': 'emocaopositiva',
                   ':d': 'emocaopositiva',
                   ':(': 'emocaonegativa'}
  for emocao in lista_emocoes:
    texto = texto.replace(emocao, lista_emocoes[emocao])

  # Lematização
  documento = nlp(texto)

  lista = []
  for token in documento:
    lista.append(token.lemma_)
  
  # Usamos uma compressão de lista para eliminar Stop words e pontuações
  lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in string.punctuation]
  lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])
  
  return lista



Observe que a lematização considera as pontuações e as stop words. Por isso utilizamos o método puntuaction da biblioteca string para filtrar esses elementos em conjunto com o método isdigit() do python para filtar os números.


In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
"3".isdigit()

True

In [ ]:
"três".isdigit()

False

## AGORA VAMOS APLICAR NOSSA FUNÇÃO NA BASE DE DADOS

In [ ]:
df

,tweet_text,sentiment
0,@Laranjito76 A pessoa certa para isso seria o ...,1
1,"@behin_d_curtain Para mim, é precisamente o co...",1
2,Vou fazer um video hoje... estou pensando em f...,1
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1
4,Valoriza o coração do menininho que vc tem. El...,1
...,...,...
49995,:( é tão lindo que dói https://t.co/GqnpgyWWxB,0
49996,"@veraluciarj Pois é.. tenho problema c/ ""coisa...",0
49997,eu te amo tanto minja vidinha meu bem mais pre...,0
49998,@itsLary @jessboluda Pfvor :(,0


In [ ]:
df['tweet_text'] = df['tweet_text'].apply(preprocessamento)

In [ ]:
df

,tweet_text,sentiment
0,o pessoa certo parir seriar o valer e azeved...,1
1,parir mim precisamente o contrário emocaopos...,1
2,video hoje ... pensar falar sobrar o meter csg...,1
3,aaaaaaaa amar polaroids expressar o quantum to...,1
4,valorizar o coração menino vc diferente o sorr...,1
...,...,...
49995,emocaonegativa lindar doer,0
49996,.. problema c/ .. toc emocaonegativa o entan...,0
49997,amar minja vida precioso desculpar umar mau ut...,0
49998,pfvor emocaonegativa,0
